In [ ]:
# This makes a cube of FITS files, so as to inspect images en masse with DS9

# Saved as .py file, 2017 Nov 19

# created 2017 Nov 19 by E.S.

In [1]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import numpy as np
import pandas as pd
import datetime
from astropy.io import fits
import asciitable
from pathlib import Path
import pickle
import astropy
from astropy import units as u
from astropy.coordinates import SkyCoord

In [2]:
# set file paths and datasets

dirTreeStem = ('/home/../../media/unasemaje/Seagate Expansion Drive/lbti_data_reduction/'
               'lmircam_astrometry/ut_2017_11_06/asterism/processedData/')
retrievalPiece = ('step03_derotate/')
depositPiece = ('step04_ditherMedians/')
fileNameStem = ('lm_171106_')

In [3]:
picklefile = 'identified_stars_171106.p'
fo1=open(picklefile,'rb')
dat=pickle.load(fo1)
fo1.close()

In [5]:
# read in text file

df = pd.read_csv('star_ra_dec_list.csv')

In [7]:
# extract RA, DEC for C1/2 and D1

chooseStar1 = np.where(df[' shorthand']==' '+dat['dither_pos_00'][1][0])[0]
chooseStar2 = np.where(df[' shorthand']==' '+dat['dither_pos_00'][1][2])[0]
raString1 = df[' RA'][chooseStar1].values
decString1 = df[' DEC'][chooseStar1].values
raString2 = df[' RA'][chooseStar2].values
decString2 = df[' DEC'][chooseStar2].values

In [23]:
c_C12 = SkyCoord(raString1+decString1, unit=(u.hourangle, u.deg))
c_zeta = SkyCoord(raString2+decString2, unit=(u.hourangle, u.deg))
c_C12.position_angle(c_zeta).degree

array([ 33.35034613])

In [29]:
raString+decString

array([' 5:35:16.4602 -5:23:22.8832'], dtype=object)

In [30]:
c3test = SkyCoord(raString+decString, unit=(u.hourangle, u.deg))

In [34]:
print(c3)
print(c3test)

<SkyCoord (ICRS): (ra, dec) in deg
    ( 83.81552292, -5.38956519)>
<SkyCoord (ICRS): (ra, dec) in deg
    [( 83.81858417, -5.38968978)]>


In [15]:
str(df[' RA'][1])+str(df[' DEC'][1])

' 5:35:15.7673 -5:23:9.82764'

In [33]:
c3 = SkyCoord(str(df[' RA'][0])+str(df[' DEC'][0]), unit=(u.hourangle, u.deg))
c4 = SkyCoord(str(df[' RA'][1])+str(df[' DEC'][1]), unit=(u.hourangle, u.deg))
c5 = SkyCoord(str(0)+str(df[' DEC'][1]), unit=(u.hourangle, u.deg))
c3.position_angle(c4).degree

2.834654188645067

In [22]:
print(c4)
print(c5)
print(c5.position_angle(c4).degree)

<SkyCoord (ICRS): (ra, dec) in deg
    ( 83.81569708, -5.38606323)>
<SkyCoord (ICRS): (ra, dec) in deg
    ( 0., -5.38606323)>
94.815483148266


In [26]:
c1 = SkyCoord(0*u.deg, 0*u.deg)
c2 = SkyCoord(-1*u.deg, 0*u.deg)
c1.position_angle(c2).degree

270.0

In [28]:
print(c1.dec)
print(c2.dec)

0d00m00s
0d00m00s


In [7]:
df

,name,shorthand,RA,DEC
0,Parenago 1866,gamma,5:35:15.7255,-5:23:22.4347
1,tet01 Ori E,E1,5:35:15.7673,-5:23:9.82764
2,JW489,epsilon,5:35:15.7879,-5:23:26.5168
3,tet01 Ori A1,A1,5:35:15.8202,-5:23:14.2891
4,Parenago 1867,beta,5:35:15.8337,-5:23:22.4207
5,TCC47,delta,5:35:15.8408,-5:23:25.5078
6,JW494,omega,5:35:15.8739,-5:23:1.89992
7,tet01 Ori B2,B2,5:35:16.069,-5:23:6.96452
8,tet01 Ori B1,B1,5:35:16.1299,-5:23:6.71895
9,LV 3,theta,5:35:16.283,-5:23:16.512


In [ ]:
coordsModel_d = np.array(dat['coordsModel_d_matched']) # not sure what this would be useful for, though
coordsModel_not_d = np.array(dat['coordsModel_not_d_matched']) # 'ideal' (x,y)
coordsEmpirical =  np.array(dat['coordsEmpirical_matched']) # empirical (x,y)

In [3]:
def make_median(ditherPos,startNum,stopNum,xSize=2048,ySize=2048,N=1):
    '''
    makes a median from a range of FITS files
    INPUTS--
    ditherPos: number indicating the dither number
    startNum: first frame number
    stopNum: last frame number (inclusive)
    xSize: pixels in x-dimension
    ySize: pixels in y-dimension
    N: number of frames between each one used to make the median (default is 1: no skipped frames)
    '''
    N = N # use every N frames
    xSize = xSize
    ySize = ySize
    sliceNum = 0 # initialize cube slice number
    cubeArray = np.zeros((int(np.divide(stopNum-startNum+1,N)),xSize,ySize),dtype=np.int64)
    for f in range(startNum,stopNum+1): # loop over filenames
        if (f%N == 0): 
            print('----------------------------------------')
            print(str(f))
            image, header = fits.getdata(dirTreeStem+
                                         retrievalPiece+
                                 fileNameStem+
                                 str("{:0>5d}".format(f))+
                                 '.fits',
                                 0,
                                 header=True)
            
            cubeArray[sliceNum,:,:] = image
            sliceNum += 1 # move to next slice
    
    # take median
    medianArray = np.median(cubeArray,axis=0)
    
    # save median
    hdu = fits.PrimaryHDU(medianArray)
    hdulist = fits.HDUList([hdu])
    hdulist.writeto(dirTreeStem+depositPiece+fileNameStem+'dither_'+str("{:0>2d}".format(ditherPos))+'.fits',
                    overwrite=False)

In [20]:
# run the function

make_median(16,8923,8941)

----------------------------------------
8923
----------------------------------------
8924
----------------------------------------
8925
----------------------------------------
8926
----------------------------------------
8927
----------------------------------------
8928
----------------------------------------
8929
----------------------------------------
8930
----------------------------------------
8931
----------------------------------------
8932
----------------------------------------
8933
----------------------------------------
8934
----------------------------------------
8935
----------------------------------------
8936
----------------------------------------
8937
----------------------------------------
8938
----------------------------------------
8939
----------------------------------------
8940
----------------------------------------
8941
